# Consultando o banco de dados com um assistente

Usamos a biblioteca [sqlalchemy](https://docs.sqlalchemy.org/en/20/) para trabalhar com bancos de dados no ambiente Python.
- O [`create_engine`](https://docs.sqlalchemy.org/en/20/core/engines.html) cria a conexão com o banco de dados usando a URL fornecida.
- O [`MetaData`](https://docs.sqlalchemy.org/en/20/core/metadata.html) gerencia informações sobre as tabelas e seus esquemas (metadados).

In [ ]:
from sqlalchemy import create_engine, MetaData

url = '/content/ecommerce.db'
engine = create_engine(f'sqlite:///{url}')

In [ ]:
metadata_obj = MetaData()
metadata_obj.reflect(engine)
metadata_obj.tables.keys()

* [`pd.read_sql_table`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_table.html) carrega a tabela diretamente para um DataFrame do pandas

In [ ]:
import pandas as pd

pd.read_sql_table('Clientes', engine)

In [ ]:
pd.read_sql_table('Fornecedores', engine)

In [ ]:
pd.read_sql_table('Funcionarios', engine)

## Configurando o Llama Index

In [ ]:
!pip install llama-index llama-index-llms-groq llama-index-experimental llama-index-embeddings-huggingface llama-index-postprocessor-cohere-rerank gradio

In [ ]:
from google.colab import userdata
key = userdata.get('GROQ_API')

Informações sobre os modelos:
- [`llama-3.1-70b-versatile`](https://github.com/meta-llama/llama-models/blob/main/models/llama3_1/MODEL_CARD.md)
- [`BAAI/bge-m3`](https://huggingface.co/BAAI/bge-m3)

In [ ]:
modelo="llama-3.1-70b-versatile"
modelo_hf_emb="BAAI/bge-m3"

In [ ]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Groq(model=modelo, api_key = key)
Settings.embed_model = HuggingFaceEmbedding(model_name = modelo_hf_emb)

- `SQLDatabase` conecta ao banco de dados SQL.
- `SQLTableNodeMapping` cria um mapeamento do banco de dados.

In [ ]:
from llama_index.core import SQLDatabase
from llama_index.core.objects import SQLTableNodeMapping

sql_database = SQLDatabase(engine)
table_node_map = SQLTableNodeMapping(sql_database)

* `SQLTableSchema` representa cada tabela no banco como um objeto e podem conter informações sobre os nomes das tabelas e, eventualmente, descrições.

In [ ]:
from llama_index.core.objects import SQLTableSchema

table_schema_objs = []
for nome_tabelas in metadata_obj.tables.keys():
  table_schema_objs.append(SQLTableSchema(table_name=nome_tabelas))

* `ObjectIndex` combina os objetos de esquema (SQLTableSchema) e o mapeamento (SQLTableNodeMapping) para construir um índice vetorial com embeddings.

In [ ]:
from llama_index.core.objects import ObjectIndex
from llama_index.core import VectorStoreIndex

obj_index = ObjectIndex.from_objects(table_schema_objs,table_node_map, VectorStoreIndex)

* `retriever` é um objeto usado para encontrar a tabela ou objeto mais relevante no banco.

In [ ]:
obj_retriever = obj_index.as_retriever(similarity_top_k=1)

## Consultando o banco de dados

* `SQLTableRetrieverQueryEngine` interpreta e executa consultas SQL com base em entrada de linguagem natural.

In [ ]:
from llama_index.core.indices.struct_store.sql_query import SQLTableRetrieverQueryEngine

query_engine = SQLTableRetrieverQueryEngine(sql_database, obj_retriever)

1° consulta

```
'Quais estados mais frequentes na tabela Clientes?'
```

In [ ]:
resposta = query_engine.query('Quais estados mais frequentes na tabela Clientes?')

In [ ]:
resposta

In [ ]:
print(resposta)

In [ ]:
resposta.metadata

2° consulta

```
'Quais as fornecedoras localizadas na cidade de São Paulo?'
```

In [ ]:
resposta = query_engine.query('Quais as fornecedoras localizadas na cidade de São Paulo?')
print(resposta)

3° consulta

```
'Quantos funcionários com o cargo de vendedores temos cadastrados?'
```

In [ ]:
resposta = query_engine.query('Quantos funcionários com o cargo de vendedores temos cadastrados?')
print(resposta)

# Contextualizando o modelo

Utilizando LLM do Groq com LlamaIndex
- [Guia Groq](https://docs.llamaindex.ai/en/stable/examples/llm/groq/)

In [ ]:
llm = Groq(model=modelo, api_key = key)

In [ ]:
def gerar_descricao_tabela(nome_tabela, df_amostra):
    prompt = f"""
    Analise a amostra da tabela '{nome_tabela}' abaixo e forneça uma curta e breve descrição do conteúdo dessa tabela.
    Informe até o máximo de 5 valores únicos de cada coluna.


    Amostra da Tabela:
    {df_amostra}

    Descrição:
    """

    resposta = llm.complete(prompt = prompt)

    return resposta.text

In [ ]:
nomes_tabelas = metadata_obj.tables.keys()
dicionario_tabelas = {}

for nome_tabela in nomes_tabelas:
  df = pd.read_sql_table(nome_tabela,engine)
  df_amostra = df.head(5).to_string()

  descricao = gerar_descricao_tabela(nome_tabela, df_amostra)
  dicionario_tabelas[nome_tabela] = descricao
  print(f'Tabela: {nome_tabela}\n Descrição: {descricao}')
  print('-'*15)

In [ ]:
table_schema_objs = [
    SQLTableSchema(table_name= nome_tabela,context_str= dicionario_tabelas[nome_tabela])
    for nome_tabela in nomes_tabelas
]

A descrição das tabelas precisa ser adicionada no schema das tabelas

In [ ]:
obj_index = ObjectIndex.from_objects(table_schema_objs,table_node_map, VectorStoreIndex)
obj_retriever = obj_index.as_retriever(similarity_top_k=1)

In [ ]:
query_engine_2 = SQLTableRetrieverQueryEngine(sql_database,obj_retriever)

1° consulta com nova `query_engine`

```
'Quais as fornecedoras localizadas na cidade de São Paulo?'
```

In [ ]:
resposta = query_engine_2.query('Quais as fornecedoras localizadas na cidade de São Paulo?')
print(resposta)

2° consulta com nova `query_engine`

```
'Quantos funcionários com o cargo de vendedores temos cadastrados?'
```

In [ ]:
resposta = query_engine_2.query('Quantos funcionários com o cargo de vendedores temos cadastrados?')
print(resposta)

# Iniciando a personalização de prompts

**Estrutura da cadeia passos**

![Alt text: O diagrama inicia com uma "Entrada" indicada por um círculo na cor verde, simbolizando que a tarefa está feita, que leva a várias etapas sequenciais e decisionais.
Primeiramente, a entrada conecta-se a uma etapa chamada "Acesso à tabela", seguida por "Contexto da tabela" e depois para o "1º Prompt". Estas etapas parecem preparar a consulta inicial, contextualizando-a antes de processá-la através de um modelo de linguagem de aprendizado profundo (LLM), indicado no fluxo.
Após o LLM processar o primeiro prompt, os resultados são utilizados para obter uma "Consulta SQL" e, a partir dela, o "Resultado Consulta SQL". O fluxo prossegue para um "2º Prompt (Síntese)", sugerindo uma etapa de elaboração ou síntese baseada no resultado e a consulta SQL.
Finalmente, a saída deste segundo prompt aplicando em uma LLM leva à "Resposta final", indicando a conclusão do processo.](https://github.com/Mirlaa/llamaindex-assistente-BD/blob/main/Suporte-educacional/esquema-Assitente-BD-Zoop.png?raw=true)

## Criação do 1° Prompt

In [ ]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
print(DEFAULT_TEXT_TO_SQL_PROMPT.template)

In [ ]:
texto2sql = """Dada uma pergunta em linguagem natural, crie uma consulta {dialect} sintaticamente correta para executar e, em seguida, verifique os resultados da consulta e retorne a resposta. Você pode ordenar os resultados por uma coluna relevante para retornar os exemplos mais informativos no banco de dados.

Nunca consulte todas as colunas de uma tabela específica. Pergunte apenas por algumas colunas relevantes, de acordo com a pergunta.

Preste atenção para usar apenas os nomes de colunas que você pode ver na descrição do esquema. Tenha cuidado para não consultar colunas que não existem. Preste atenção em qual coluna está em qual tabela. Além disso, qualifique os nomes das colunas com o nome da tabela quando necessário.

Use o seguinte formato, cada um em uma linha:

Pergunta: Pergunta aqui
ConsultaSQL: Consulta SQL para executar
ResultadoSQL: Resultado da ConsultaSQL
Resposta: Resposta final aqui

Use apenas as tabelas listadas abaixo.

{schema}

Pergunta: {pergunta_user}
ConsultaSQL:
"""

* `PromptTemplate` permite criar um template de um prompt para ser enviado à uma LLM.

In [ ]:
from llama_index.core import PromptTemplate

prompt_1 = PromptTemplate(texto2sql, dialect = engine.dialect.name)
print(prompt_1.template)

## Descrição tabelas

In [ ]:
from typing import List

def descricao_tabela (schema_tabelas: List[SQLTableSchema]):
  descricao_str = []
  for tabela_schema in schema_tabelas:
    info_tabela = sql_database.get_single_table_info(tabela_schema.table_name)
    info_tabela += (' A descrição da tabela é: '+tabela_schema.context_str)

    descricao_str.append(info_tabela)
  return '\n\n'.join(descricao_str)

* `FnComponent` é uma classe do LlamaIndex usada para encapsular funções como componentes reutilizáveis dentro de uma Query Pipeline.

In [ ]:
from llama_index.core.query_pipeline import FnComponent

contexto_tabela = FnComponent(fn=descricao_tabela)

## Consulta SQL e resultado

In [ ]:
from llama_index.core.llms import ChatResponse

def resposta_sql (resposta: ChatResponse) -> str:
  conteudo_resposta = resposta.message.content

  sql_consulta = conteudo_resposta.split("ConsultaSQL: ", 1)[-1].split("ResultadoSQL: ", 1)[0]
  return sql_consulta.strip().strip('```').strip()

consulta_sql = FnComponent(fn=resposta_sql)

* `SQLRetriever` executa consultas SQL diretamente no banco de dados configurado.

In [ ]:
from llama_index.core.retrievers import SQLRetriever

resultado_sql = SQLRetriever(sql_database)

# Construindo a cadeia de ações

**CRIAÇÃO DO 2° PROMPT**

In [ ]:
prompt_2_str = '''
    Você é o "Assitente de consulta de banco de dados da Zoop".
    Dada a seguinte pergunta, a consulta SQL correspondente e o resultado SQL, responda à pergunta de modo agradável e objetivamente.
    Evite iniciar conversas com cumprimentos e apresentações, como "Olá".

    Pergunta: {pergunta_user}
    Consulta SQL: {consulta}
    Resultado SQL: {resultado}
    Resposta:
    '''

prompt_2 = PromptTemplate(
    prompt_2_str,
)

**DEFINIÇÃO DOS MÓDULOS DA ESTRUTURA**

In [ ]:
from llama_index.core.query_pipeline import QueryPipeline, InputComponent

qp = QueryPipeline(
    modules = {
        'entrada': InputComponent(),
        'acesso_tabela': obj_retriever,
        'contexto_tabela': contexto_tabela,
        'prompt_1': prompt_1,
        'llm_1': llm,
        'consulta_sql': consulta_sql,
        'resultado_sql': resultado_sql,
        'prompt_2': prompt_2,
        'llm_2': llm,
    },
    verbose=False
)

## Definindo conexão de ações

**Vamos usar nossa estrutura de apoio:**

![Alt text: O diagrama inicia com uma "Entrada" indicada por um círculo na cor verde, simbolizando que a tarefa está feita, que leva a várias etapas sequenciais e decisionais.
Primeiramente, a entrada conecta-se a uma etapa chamada "Acesso à tabela", seguida por "Contexto da tabela" e depois para o "1º Prompt". Estas etapas parecem preparar a consulta inicial, contextualizando-a antes de processá-la através de um modelo de linguagem de aprendizado profundo (LLM), indicado no fluxo.
Após o LLM processar o primeiro prompt, os resultados são utilizados para obter uma "Consulta SQL" e, a partir dela, o "Resultado Consulta SQL". O fluxo prossegue para um "2º Prompt (Síntese)", sugerindo uma etapa de elaboração ou síntese baseada no resultado e a consulta SQL.
Finalmente, a saída deste segundo prompt aplicando em uma LLM leva à "Resposta final", indicando a conclusão do processo.](https://github.com/Mirlaa/llamaindex-assistente-BD/blob/main/Suporte-educacional/esquema-Assitente-BD-Zoop.png?raw=true)

In [ ]:
qp.add_chain(['entrada', 'acesso_tabela', 'contexto_tabela'])
qp.add_link('entrada', 'prompt_1', dest_key='pergunta_user')
qp.add_link('contexto_tabela', 'prompt_1', dest_key='schema')
qp.add_chain(['prompt_1', 'llm_1', 'consulta_sql', 'resultado_sql'])
qp.add_link('entrada', 'prompt_2', dest_key='pergunta_user')
qp.add_link('consulta_sql', 'prompt_2', dest_key= 'consulta')
qp.add_link('resultado_sql', 'prompt_2', dest_key='resultado')
qp.add_link('prompt_2', 'llm_2')

## Testando as saídas

1° consulta

```
'Quais estados mais frequentes na tabela Clientes?'
```

In [ ]:
resposta = qp.run(query='Quais estados mais frequentes na tabela Clientes?')
print(str(resposta))

2° consulta

```
'Quantos funcionários com o cargo de vendedores temos cadastrados?'
```

In [ ]:
resposta = qp.run(query='Quantos funcionários com o cargo de vendedores temos cadastrados?')
print(str(resposta))

# Criando um chat com a IA

**Função para executar uma consulta**

In [ ]:
def entrada_saida (msg_user: str):
  saida = qp.run(query=msg_user)
  return str(saida.message.content)

**Função para adicionar a resposta da IA ao histórico de conversa**

In [ ]:
def adicao_historico(msg_user, historico):
  msg_assistente = entrada_saida(msg_user)

  historico.append([msg_user, msg_assistente])
  return msg_assistente, historico

**Iniciar a construção da interface**

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown('## Chat com Assistente SQL')
    gr.Markdown(
        '''
        Este é um assistente SQL interativo, projetado para responder perguntas sobre os dados da loja Zoop.
        Insira sua pergunta no campo abaixo e o assistente irá responder com base no resultado da consulta SQL
        nos dados disponíveis.
        '''
    )
    chatbot = gr.Chatbot(label='Chat com Assistente')
    msg = gr.Textbox(label='Digite a sua pergunta e tecle Enter para enviar',
                     placeholder='Digite o texto aqui.')
    limpeza = gr.Button('Limpar a conversa')
    def atualizar_historico (msg_user, historico):
      msg_assistente, historico = adicao_historico(msg_user, historico)
      return '', historico
    msg.submit(atualizar_historico, inputs=[msg, chatbot], outputs=[msg, chatbot], queue= False)
    limpeza.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

demo.queue()
demo.launch(debug=True)